In [65]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import regex as re
import random

from data.keyvault import keyvault

In [2]:
###########
# Settings
###########

DATA_PATH = "data/Otvety.txt"
ANSWERS_PATH = 'data/prepared_answers.txt'
FASTTEXT_MODEL_PATH = 'data/ft_100_model.h5'
RAW_SENTENCES_PATH = 'data/raw_sentences.pkl'
INDEX_MAP_PATH = 'data/index_map.pkl'
FT_INDEX_PATH = 'data/ft_index'
FT_VECTOR_MAX_LEN = 100

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

## Рутина предобработки. Подготовка и обучение FastText модели на ответах mail.ru

In [6]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
def get_sentences(input_file, len_limit=1500000):
    
    """Process text. extract sentense tokens with morph processor"""
    
    # Preprocess for models fitting

    sentences = []

    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    c = 0

    with open(input_file, "r", encoding='utf-8') as fin:
        for line in tqdm(fin):
            spls = preprocess_txt(line, exclude, morpher, sw)
            sentences.append(spls)
            c += 1
            if c > len_limit:
                break
                
    return sentences

In [6]:
sentences = get_sentences(DATA_PATH)

In [14]:
modelFT = FastText.load(FASTTEXT_MODEL_PATH)

In [20]:
# modelFT = FastText.load("ft_model.h5")

# Создание индексов FastText
ft_index = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN ,'angular')

index_map = {}
counter = 0

with open(ANSWERS_PATH, "r", encoding='utf-8') as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(FT_VECTOR_MAX_LEN)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

# Сохранение просчитанной карты индексов
with open(INDEX_MAP_PATH, 'wb') as f:
    pickle.dump(index_map, f)
    
# Построеиние индексов на 20 деревьях и сохранение на диск
ft_index.build(20)
ft_index.save(FT_INDEX_PATH)

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## Реализация функционала. Загрузка и тестирование бота

In [3]:
# Загрузка заготовок: Fast text модель, индексы, индексированная карта ответов

modelFT = FastText.load(FASTTEXT_MODEL_PATH)

ft_index = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN, 'angular')
ft_index.load(FT_INDEX_PATH)

with open(INDEX_MAP_PATH, 'rb') as f:
    index_map = pickle.load(f)

In [7]:
def preprocess_txt(line):
    """Строковый препроцессинг,  текстовой строки строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

def get_response(question, index, model, index_map):
    
    """Полечение ответа из модели fast text"""
    
    question = preprocess_txt(question)
    vector = np.zeros(FT_VECTOR_MAX_LEN)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [90]:
# тестирование модуля болталки на "ответах mail.ru"
TEXT = "есть ли жизнь на марсе?"
r = get_response(TEXT, ft_index, modelFT, index_map)
r

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


['точно есть. везде есть. а мы вообще на земле являемся подопытными кроликами.. \n',
 'пока еще нет)) но скоро и там появяться армяне и китайцы)) и будут продавать чебуреки и шмотки)))))). \n',
 'Доброго!!! !<br>Есть там жизнь-нет там жизни нам это неизвестно, но опыты по производству жизни ведуться. \n']

In [92]:
clear_sent(r)

'пока еще нет)) но скоро и там появяться армяне и китайцы)) и будут продавать чебуреки и шмотки)))))). '

## Запуск бота

In [107]:
updater = Updater(token=keyvault['telegram_token'])
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = keyvault['google_api_key'] # Ключ к google API dialogflow

DIALOGFLOW_PROJECT_ID = keyvault['dialogflow_project_id'] # PROJECT ID DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # языковая группа
SESSION_ID = 'SetMyTasksBot'  # ID телеграмм бота

start_msg = 'Чат-бот R1D0 приветствует тебя. Я умею поддерживать беседу и отвечать на вопросы'

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


In [103]:
def clear_sent(text):
    target_sent = random.choice(text)
    target_sent = re.sub(r'<[^>]*>','', target_sent)
    target_sent = re.sub(r'[\xa0]',' ', target_sent)
    target_sent = re.sub(r'[\n]','', target_sent)
    target_sent = re.sub(r'\s[.]\s','', target_sent)
    return target_sent

In [104]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)

def textMessage(bot, update):
    
    # Начало: Поиск интентов на DialogFlow
    input_txt = update.message.text
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=input_txt, language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text and text != 'null':
        bot.send_message(chat_id=update.message.chat_id, text= response.query_result.fulfillment_text)
    else:
        # Интент DialogFlow на найден --> включение режима болталки на "ответах mail.ru"
        answers_response = get_response(input_txt, ft_index, modelFT, index_map)
        answers_response = clear_sent(answers_response)
        bot.send_message(chat_id=update.message.chat_id, text=answers_response)

In [105]:
# Хендлеры

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)

# Добавляем хендлеры в диспатчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота на Ctrl + C
updater.idle()